In [1]:
%load_ext autotime
# %unload_ext autotime

In [2]:
import sys
sys.path.append("..")

In [3]:
import numpy as np
import parselmouth
import pyaudio
from sonorus.audio import VADAudioInputStreamer
from sonorus.audio.utils import audio_float2int
from sonorus.audio.praat import (reduce_noise, 
                                 change_gender, 
                                 change_pitch)
from std_msgs.msg import UInt8MultiArray
from IPython.display import Audio
import librosa
import soundfile as sf
import ipywidgets
import pickle
import wave
from pathlib import Path

In [4]:
male_file = "/home/harold/Downloads/chapter-33.wav"
# male_file = "/home/harold/Downloads/long.wav"
# male_file = "/home/harold/Downloads/coffee-01.wav"

In [5]:
audio, sr = librosa.load(male_file, sr=24000, dtype=np.float64)
male_sound = parselmouth.Sound(values=audio, sampling_frequency=sr)
print(male_sound.values, male_sound.sampling_frequency)
Audio(data=male_sound.values, 
      rate=male_sound.sampling_frequency)

[[-0.01171875 -0.00161743  0.00921631 ... -0.00143433 -0.00161743
  -0.0017395 ]] 24000.0


In [6]:
noise_reduced_sound = male_sound
# noise_reduced_sound = reduce_noise(male_sound)
# print(noise_reduced_sound.values, 
#       noise_reduced_sound.sampling_frequency)
# Audio(data=noise_reduced_sound.values, 
#       rate=noise_reduced_sound.sampling_frequency)

In [7]:
female_sound = change_gender(noise_reduced_sound, 
                             formant_shift_ratio=1.12,
                             pitch_range_factor=1.0)
print(female_sound.values, 
      female_sound.sampling_frequency)
Audio(data=female_sound.values, 
      rate=female_sound.sampling_frequency)

# def interactive_change_gender(sound_file=male_file, factor=1.2):
#     sound = parselmouth.Sound(sound_file)
#     sound_changed_gender = change_gender(sound, formant_shift_ratio=factor)
#     return Audio(data=sound_changed_gender.values, rate=sound_changed_gender.sampling_frequency)

# w = ipywidgets.interact(interactive_change_gender,
#                         factor=ipywidgets.FloatSlider(min=0.25, 
#                                                       max=4, 
#                                                       step=0.05, 
#                                                       value=1.2))

[[-7.89038107e-07  9.47249360e-07 -7.26013085e-07 ...  3.55613321e-07
  -1.84319322e-07  2.22725775e-07]] 24000.0


In [8]:
pitch_sound = change_pitch(noise_reduced_sound, 
                           factor=1.5)
print(pitch_sound.values, 
      pitch_sound.sampling_frequency)
Audio(data=pitch_sound.values, 
      rate=pitch_sound.sampling_frequency)

# def interactive_change_pitch(sound_file=male_file, factor=1.5):
#     sound = parselmouth.Sound(sound_file)
#     sound_changed_pitch = change_pitch(sound, factor)
#     return Audio(data=sound_changed_pitch.values, rate=sound_changed_pitch.sampling_frequency)

# w = ipywidgets.interact(interactive_change_pitch,
#                         factor=ipywidgets.FloatSlider(min=0.25, 
#                                                       max=4, 
#                                                       step=0.05, 
#                                                       value=1.5))

[[-0.01169488 -0.00159356  0.00924018 ... -0.00141045 -0.00159356
  -0.00171563]] 24000.0


In [9]:
# sf.write(
#     file=male_file[:-4]+"-pitch-changed.wav",
#     data=audio_float2int(pitch_sound.values)[0],
#     samplerate=sr,
# )

In [10]:
from kaldi.feat.mfcc import Mfcc, MfccOptions
from kaldi.feat.mel import MelBanksOptions
from kaldi.matrix import Vector
from kaldi.gmm import DiagGmm
from kaldi.base.io import istream, ifstream

In [11]:
mel_opts = MelBanksOptions(num_bins=40)
mel_opts.low_freq = 20
mel_opts.high_freq = -400

mfcc_opts = MfccOptions()
mfcc_opts.num_ceps = 40
mfcc_opts.mel_opts = mel_opts
mfcc_opts.use_energy = False

mfcc = Mfcc(mfcc_opts)

In [12]:
mfcc.compute(wave=Vector(pitch_sound.values[0]), vtln_warp=1)


 3.2051e+01 -4.3159e+01 -8.1752e+00  ...  -2.8173e+00 -4.2473e-01  3.1072e+00
 3.3210e+01 -4.4194e+01 -1.5798e+01  ...  -2.2159e+00  7.3852e-01 -1.3971e-01
 3.2652e+01 -4.4653e+01 -8.7988e+00  ...   1.9063e+00  2.2161e-01 -9.1362e-01
                ...                   ⋱                   ...                
 3.3054e+01 -4.5621e+01 -1.0369e+01  ...  -3.9961e-01 -5.2204e+00 -2.2753e+00
 3.3874e+01 -4.2159e+01 -1.0995e+01  ...   7.3188e-01 -7.4643e+00 -3.0499e+00
 3.2454e+01 -4.5156e+01 -1.0835e+01  ...  -4.3573e+00 -1.5550e+00  9.9739e-01
[kaldi.matrix._matrix.Matrix of size 181x40]

In [13]:
# gmm_path = "/home/harold/Documents/IISc-work/"\
#             "local-speech-generation/kaldi/s5/"\
#             "exp/nnet3_cleaned/extractor/final.dubm"
# stream = ifstream.from_file(gmm_path)
# dubm = DiagGmm.read(stream)

In [14]:
from sonorus.speech.kaldi.create_confs import (create_mfcc_conf,
                                               create_ivector_extractor_conf)

from sonorus.speech.kaldi import PhonemeSegmenter

In [15]:
phoneme_segmenter = PhonemeSegmenter.from_url(force_download=False)

Model from https://www.dropbox.com/s/eecyok5h1pfn7vy/librispeech_tgsmall.tar.gz?dl=1 already downloaded, cached model will be used.


In [16]:
wav_file = "/home/harold/Pictures/data/test/chapter-33-pitch-changed.wav"
wav_scp = "/home/harold/Pictures/data/test/wav.scp"
spk2utt = "/home/harold/Pictures/data/test/spk2utt"

In [17]:
seg = phoneme_segmenter.segment(pitch_sound.values, sample_rate=sr)
seg

{'utt1': {'text': 'CHAPTER THIRTY THREE',
  'phonemes': [{'name': 'SIL', 'start': 0.0, 'duration': 0.061},
   {'name': 'CH', 'start': 0.061, 'duration': 0.0305},
   {'name': 'AE', 'start': 0.0915, 'duration': 0.0915},
   {'name': 'P', 'start': 0.183, 'duration': 0.061},
   {'name': 'T', 'start': 0.244, 'duration': 0.061},
   {'name': 'ER', 'start': 0.305, 'duration': 0.061},
   {'name': 'TH', 'start': 0.366, 'duration': 0.122},
   {'name': 'ER', 'start': 0.488, 'duration': 0.061},
   {'name': 'D', 'start': 0.549, 'duration': 0.0915},
   {'name': 'IY', 'start': 0.6405, 'duration': 0.0305},
   {'name': 'TH', 'start': 0.671, 'duration': 0.122},
   {'name': 'R', 'start': 0.793, 'duration': 0.122},
   {'name': 'IY', 'start': 0.915, 'duration': 0.2745},
   {'name': 'SIL', 'start': 1.1895, 'duration': 0.0305}]}}

In [18]:
from hr.lip_control import PhonemesPublisher

phoneme_pub = PhonemesPublisher()
phoneme_pub.to_visemes(seg["utt1"]["phonemes"])

/home/harold/Documents/IISc-work/sonorus/examples/hr/actuator/names.py:23: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  head_actuators = ACTUATOR_NAMES.str.contains(head_actuator_regex, regex=True, na=False)


[{'name': 'Sil', 'start': 0.0, 'duration': 0.061},
 {'name': 'C-D-G-K-N-S-TH', 'start': 0.061, 'duration': 0.0305},
 {'name': 'A-I', 'start': 0.0915, 'duration': 0.0915},
 {'name': 'M', 'start': 0.183, 'duration': 0.061},
 {'name': 'C-D-G-K-N-S-TH', 'start': 0.244, 'duration': 0.061},
 {'name': 'C-D-G-K-N-S-TH', 'start': 0.305, 'duration': 0.061},
 {'name': 'C-D-G-K-N-S-TH', 'start': 0.366, 'duration': 0.122},
 {'name': 'C-D-G-K-N-S-TH', 'start': 0.488, 'duration': 0.061},
 {'name': 'C-D-G-K-N-S-TH', 'start': 0.549, 'duration': 0.0915},
 {'name': 'E', 'start': 0.6405, 'duration': 0.0305},
 {'name': 'C-D-G-K-N-S-TH', 'start': 0.671, 'duration': 0.122},
 {'name': 'L', 'start': 0.793, 'duration': 0.122},
 {'name': 'E', 'start': 0.915, 'duration': 0.2745},
 {'name': 'Sil', 'start': 1.1895, 'duration': 0.0305}]

In [19]:
from sonorus.audio import VADAudioInputStreamer

In [20]:
audio_streamer = VADAudioInputStreamer(
    pa_format=pyaudio.paInt16, 
)

with audio_streamer as streamer:
        for stream in streamer.stream():
            if stream is not None:
                audio = np.frombuffer(stream, dtype=np.int16)
                print(audio.shape, audio[:5])
                phonemes = phoneme_segmenter.segment(
                    audio, sample_rate=streamer.processing_rate)
                print(phonemes)

(44480,) [ 0  0  3 -9 19]
{'utt1': {'text': '', 'phonemes': [{'name': 'SIL', 'start': 0.0, 'duration': 2.78}]}}
(27840,) [ -75 -281  -48  290  208]
{'utt1': {'text': "RANG'D IN YOUR EAR", 'phonemes': [{'name': 'SIL', 'start': 0.0, 'duration': 0.27}, {'name': 'R', 'start': 0.27, 'duration': 0.06}, {'name': 'AE', 'start': 0.33, 'duration': 0.09}, {'name': 'NG', 'start': 0.42, 'duration': 0.06}, {'name': 'D', 'start': 0.48, 'duration': 0.09}, {'name': 'IH', 'start': 0.57, 'duration': 0.12}, {'name': 'N', 'start': 0.69, 'duration': 0.06}, {'name': 'Y', 'start': 0.75, 'duration': 0.06}, {'name': 'UH', 'start': 0.81, 'duration': 0.03}, {'name': 'R', 'start': 0.84, 'duration': 0.06}, {'name': 'IY', 'start': 0.9, 'duration': 0.18}, {'name': 'R', 'start': 1.08, 'duration': 0.39}, {'name': 'SIL', 'start': 1.47, 'duration': 0.27}]}}
(59200,) [ 669  447  125 -282 -618]
{'utt1': {'text': 'EVERY HILL EH', 'phonemes': [{'name': 'SIL', 'start': 0.0, 'duration': 1.594309}, {'name': 'EH', 'start': 1.594